In [1]:
# This is for Short Volume code for Previous Months for Finra

In [2]:
import pandas as pd
from pandas import DataFrame
import requests
#import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
#import numpy as np
from openpyxl import load_workbook

In [3]:
# To find the all the weekdays in a month
month_start = input('What is the month you want to find out (Use num.)? (ie Jan=1, April=4, Dec=12, etc) ')
year_start = input('What is the year you want to find out? (ie 2021, 2020, 2019, etc) ')
month_object_start = datetime.strptime(month_start, "%m") #To convert to datetime format
year_object_start = datetime.strptime(year_start, "%Y") #To convert to datetime format

What is the month you want to find out (Use num.)? (ie Jan=1, April=4, Dec=12, etc) 4
What is the year you want to find out? (ie 2021, 2020, 2019, etc) 2021


In [4]:
from calendar import monthrange
days = monthrange(int(year_start),int(month_start)) # To find out how many days in a month

In [5]:
date_need = []
for i in range(1,days[1]+1):
    date_month = datetime(int(year_start),int(month_start),i) # To find the first day of the month
    # Below will go thru each day in day month and if weekday, will add it to date_need
    if date_month.strftime("%A") in ['Monday','Tuesday','Wednesday','Thursday','Friday']: 
        date_need.append(date_month.strftime("%Y%m%d"))
#len(date_need)

In [6]:
url = 'http://regsho.finra.org/CNMSshvol' #This is the base of the url
date_need_new = [] #All the dates that have trading days will be added to this list 
finra_month = [] # All the url which have trading will be added to this list
for item in date_need:
    urlv = url + item + '.txt' #str cannot append, so just add it
    trys = requests.get(urlv) # To get the length of the contents in url to be tested
    tryss = trys.text
    if len(tryss) > 302 : # If URL length is 302, there are no trading on that day. This will append all days got trading
        finra_month.append(urlv)
        date_need_new.append(urlv[-12:-4]) #All the dates that have trading days will be added to this list   
print(len(finra_month))

21


In [7]:
while len(finra_month) == len(date_need_new): #Just to check the quantity is correct, if not numbering will be wrong when naming
    print('No errors between the URL and dates sum')
    break
else:
    print('There is AN ERROR between the URL and dates sum')

No errors between the URL and dates sum


In [8]:
count = 0

for x in finra_month:
    response = requests.get(f'{x}')
    data = response.text
    data = data.replace('\r','') # Once URL contents are readable in python, must remove /r and /n from data
    data = data.replace('\n','|') # /n is being replaced with | due to easier when split it up below
    a = data.split('|') # All content will be split according to | for easier breakdown into columns
    del a[-2:] # Last 2 items in all data are not needed
    
    # Since each data has 6 columns, can break data up equally among 6 columns
    
    # Sum of each columns(datee,symee, etc) will be total of entire data
    # len(datee+symee+svee+sevee+tvee+mee) == len(a) : This needs to be true for code to work
    # Below is list which will contain the iterations of 6's 
    # Example >> datee will have 0,6,12,18,24,....
    # symee will have 1,7,13,19,.... 
    # can use this with below (Date, Symbol) to put a from above into list for columns in pandas
    datee = [] 
    symee = []
    svee = []
    sevee = []
    tvee = []
    mee = []
    
    # All 6 below will be columns used in pandas containing the content of the URL 
    # Date will have all the dates of the URL contents, etc
    Date = []
    Symbol = []
    Short_Volume = []
    Short_Exempt_Volume = []
    Total_Volume = []
    Market = []
    
    #This is to get all the numbers in each column to extract the data. 
    # Example >> datee will have 0,6,12,18,24,....
    # symee will have 1,7,13,19,....
    for i in range(0,6):
        for h in range(i,len(a),6):
            if i == 0:
                datee.append(h)
            elif i == 1:
                symee.append(h)
            elif i == 2:
                svee.append(h)
            elif i == 3:
                sevee.append(h)
            elif i == 4:
                tvee.append(h)
            elif i == 5:
                mee.append(h)
    
    # This will get all the data and put them into their respective columns 
    # Example for Date.append(a[dd]), a[dd] = a[01] = the first index of the a(data from URL) and so on
    for dd in datee:
        Date.append(a[dd])
    for sy in symee:
        Symbol.append(a[sy])
    for sv in svee:
        Short_Volume.append(a[sv])
    for sev in sevee:
        Short_Exempt_Volume.append(a[sev])
    for tve in tvee:
        Total_Volume.append(a[tve])
    for me in mee:
        Market.append(a[me])
    
    # Creating a blank data frame and adding Data from above with names for columns
    df = pd.DataFrame() 
    df['Date'] = Date
    df['Symbol'] = Symbol
    df['Short Volume'] = Short_Volume
    df['Short Exempt Volume'] = Short_Exempt_Volume
    df['Total Volume'] = Total_Volume
    df['Market'] = Market
    df.drop([0], inplace=True) # Drop first index
    df.reset_index(drop= True, inplace=True) # Resetting index numbers
    
    #Changing the dtype above for easier data understanding (mean, std, basically, df.describe)
    df = df.astype({"Short Volume":'int', "Short Exempt Volume":'int', "Total Volume":'int'}) 
    
    df['Date'] = pd.to_datetime(df['Date'], dayfirst = True) # change date to datetime format
    df['Date'] = df["Date"].dt.strftime('%d %B %Y') # Changing it easier readibility 
    GME = df.loc[df['Symbol'] == 'GME'] # Extracting GME numbers
    AMC = df.loc[df['Symbol'] == 'AMC'] # Extracting AMC numbers
    NIO = df.loc[df['Symbol'] == 'NIO'] # Extracting NIO numbers
    
    if count == 0: 
        # The first count or count = 0 will create a new excel file. Easier this way than referencing first day of month
        with pd.ExcelWriter(f'Short Stocks Volume {date_month.strftime("%B %Y")} - Finra.xlsx', engine='openpyxl') as writer:
                GME.to_excel(writer, sheet_name='GME')
                AMC.to_excel(writer, sheet_name='AMC')
                NIO.to_excel(writer, sheet_name='NIO')
                df.to_excel(writer, sheet_name=f'Short Volume {date_need_new[count]}')
                writer.close()
    else: 
        # Once count >= 1, it will append to exisiting file. #a for append
        with pd.ExcelWriter(f'Short Stocks Volume {date_month.strftime("%B %Y")} - Finra.xlsx', mode = 'a', engine='openpyxl') as writer:  
            # try to open an existing workbook
            writer.book = load_workbook(f'Short Stocks Volume {date_month.strftime("%B %Y")} - Finra.xlsx') 
            # copy existing sheets
            writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets) 
            # will append to existing GME sheet, sheet name here specified
            GME.to_excel(writer, 'GME',header=False,startrow=count+1) 
             # will append to existing AMC sheet, sheet name here specified
            AMC.to_excel(writer, 'AMC',header=False,startrow=count+1)
             # will create new sheet, sheet name new one given
            NIO.to_excel(writer, 'NIO',header=False,startrow=count+1)
             # will create new sheet, sheet name new one given
            df.to_excel(writer, sheet_name=f'Short Volume {date_need_new[count]}')
            writer.close() 
    
    count += 1 # Once everything is completed, count will increase by one. To help sheet naming and whether excel sheet need to append
    print(f'There are {len(finra_month)-count} URL left in this operation') # Using this to track at which part in code

There are 20 URL left in this operation
There are 19 URL left in this operation
There are 18 URL left in this operation
There are 17 URL left in this operation
There are 16 URL left in this operation
There are 15 URL left in this operation
There are 14 URL left in this operation
There are 13 URL left in this operation
There are 12 URL left in this operation
There are 11 URL left in this operation
There are 10 URL left in this operation
There are 9 URL left in this operation
There are 8 URL left in this operation
There are 7 URL left in this operation
There are 6 URL left in this operation
There are 5 URL left in this operation
There are 4 URL left in this operation
There are 3 URL left in this operation
There are 2 URL left in this operation
There are 1 URL left in this operation
There are 0 URL left in this operation
